# It definitely came as a surprise to me that there was this huge shake up in the end. 
I personally thought that the shake up should be relatively small because my stacking 
model performance on public LB and local cv was always aligned although for single model 
they could vary quite a bit. We did trust mostly on local CV and we tried very hard on nonlinear 
stacking (without success) so we relied on weighted average cv score and only submitted when there was an improvement on local cv.

# I cannot wait to see what @Michael Jahrer did but I would like to share some of our approaches (actually, just the NN part, and leave the rest to my teammates) before Michael publishes his and then nobody cares what we did :(

# So my best NN had a 5 fold local CV 0.294, and public LB 0.284 and private 0.290. And here is roughly how you can achieve that:

# 1) important features' interactions (e.g. ps_car_13, ps_ind_03, ps_reg_03, ...)

# 2) count of categorical features

# 3) xgboost predictions: divide feature sets into three groups (car, ind, reg) and then use two group as features and the other group as target, train a xgboost model on it, and use prediction as features

# 4) feature aggregation: pick two features (e.g. ps_car_13, ps_ind_03), and then use one as group variable, the other as value variable, do mean, std, max, min, median. Still top important features are picked only

# 5) do embedding layers on all categorical features (I used embedding dimension 4 with dropout 0.25)

# 6) model is 2 layer with relu activation and big dropout (512 with dropout 0.8, and 64 with 0.8)

# 7) a few categorical count features @qianqian created

# That is about it! Hope it helps :)

# Update: my best lightgbm code is here

# https://www.kaggle.com/xiaozhouwang/2nd-place-lightgbm-solution

# It scored 0.29124 on private LB.

# best nn code is here https://www.kaggle.com/xiaozhouwang/2nd-place-solution-nn-model

# It scored 0.29008 on private LB.

# And their model results can be found in the comments (Seems I cannot upload files here?)

In [1]:
# part of 2nd place solution: lightgbm model with private score 0.29124 and public lb score 0.28555

import lightgbm as lgbm
from scipy import sparse as ssp
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def Gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]

    # sort rows on prediction column
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:, 0].argsort()][::-1, 0]
    pred_order = arr[arr[:, 1].argsort()][::-1, 0]

    # get Lorenz curves
    L_true = np.cumsum(true_order) * 1. / np.sum(true_order)
    L_pred = np.cumsum(pred_order) * 1. / np.sum(pred_order)
    L_ones = np.linspace(1 / n_samples, 1, n_samples)

    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)

    # normalize to true Gini coefficient
    return G_pred * 1. / G_true

cv_only = True
save_cv = True
full_train = False

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', Gini(labels, preds), True

path = "../data/"

train = pd.read_csv(path+'train.csv')
train_label = train['target']
train_id = train['id']
test = pd.read_csv(path+'test.csv')
test_id = test['id']

NFOLDS = 5
kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=218)

y = train['target'].values
drop_feature = [
    'id',
    'target'
]

X = train.drop(drop_feature,axis=1)
feature_names = X.columns.tolist()
cat_features = [c for c in feature_names if ('cat' in c and 'count' not in c)]
num_features = [c for c in feature_names if ('cat' not in c and 'calc' not in c)]

train['missing'] = (train==-1).sum(axis=1).astype(float)
test['missing'] = (test==-1).sum(axis=1).astype(float)
num_features.append('missing')

for c in cat_features:
    le = LabelEncoder()
    le.fit(train[c])
    train[c] = le.transform(train[c])
    test[c] = le.transform(test[c])

enc = OneHotEncoder()
enc.fit(train[cat_features])
X_cat = enc.transform(train[cat_features])
X_t_cat = enc.transform(test[cat_features])

ind_features = [c for c in feature_names if 'ind' in c]
count=0
for c in ind_features:
    if count==0:
        train['new_ind'] = train[c].astype(str)+'_'
        test['new_ind'] = test[c].astype(str)+'_'
        count+=1
    else:
        train['new_ind'] += train[c].astype(str)+'_'
        test['new_ind'] += test[c].astype(str)+'_'

cat_count_features = []
for c in cat_features+['new_ind']:
    d = pd.concat([train[c],test[c]]).value_counts().to_dict()
    train['%s_count'%c] = train[c].apply(lambda x:d.get(x,0))
    test['%s_count'%c] = test[c].apply(lambda x:d.get(x,0))
    cat_count_features.append('%s_count'%c)

train_list = [train[num_features+cat_count_features].values,X_cat,]
test_list = [test[num_features+cat_count_features].values,X_t_cat,]

X = ssp.hstack(train_list).tocsr()
X_test = ssp.hstack(test_list).tocsr()


# learning_rate = 0.1
# num_leaves = 15
# min_data_in_leaf = 2000
# feature_fraction = 0.6
# num_boost_round = 10000
# params = {"objective": "binary",
#           "boosting_type": "gbdt",
#           "learning_rate": learning_rate,
#           "num_leaves": num_leaves,
#            "max_bin": 256,
#           "feature_fraction": feature_fraction,
#           "verbosity": 0,
#           "drop_rate": 0.1,
#           "is_unbalance": False,
#           "max_drop": 50,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
#           "subsample": 0.9
#           }

# x_score = []
# final_cv_train = np.zeros(len(train_label))
# final_cv_pred = np.zeros(len(test_id))
# for s in range(16):
#     cv_train = np.zeros(len(train_label))
#     cv_pred = np.zeros(len(test_id))

#     params['seed'] = s

#     if cv_only:
#         kf = kfold.split(X, train_label)

#         best_trees = []
#         fold_scores = []

#         for i, (train_fold, validate) in enumerate(kf):
#             X_train, X_validate, label_train, label_validate = \
#                 X[train_fold, :], X[validate, :], train_label[train_fold], train_label[validate]
#             dtrain = lgbm.Dataset(X_train, label_train)
#             dvalid = lgbm.Dataset(X_validate, label_validate, reference=dtrain)
#             bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dvalid, feval=evalerror, verbose_eval=100,
#                             early_stopping_rounds=100)
#             best_trees.append(bst.best_iteration)
#             cv_pred += bst.predict(X_test, num_iteration=bst.best_iteration)
#             cv_train[validate] += bst.predict(X_validate)

#             score = Gini(label_validate, cv_train[validate])
#             print(score)
#             fold_scores.append(score)

#         cv_pred /= NFOLDS
#         final_cv_train += cv_train
#         final_cv_pred += cv_pred

#         print("cv score:")
#         print(Gini(train_label, cv_train))
#         print("current score:", Gini(train_label, final_cv_train / (s + 1.)), s+1)
#         print(fold_scores)
#         print(best_trees, np.mean(best_trees))

#         x_score.append(Gini(train_label, cv_train))

# print(x_score)
# pd.DataFrame({'id': test_id, 'target': final_cv_pred / 16.}).to_csv('../cache/lgbm3_pred_avg.csv', index=False)
# pd.DataFrame({'id': train_id, 'target': final_cv_train / 16.}).to_csv('../cache/lgbm3_cv_avg.csv', index=False)

In [2]:
from numba import jit
@jit
def eval_gini(y_true, y_prob):
    """
    Original author CPMP : https://www.kaggle.com/cpmpml
    In kernel : https://www.kaggle.com/cpmpml/extremely-fast-gini-computation
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

def gini_xgb(preds, dtrn_df):
    labels = dtrn_df.get_label()
    gini_score = eval_gini(labels, preds)
    return [('gini', gini_score)]


def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))



In [3]:
# Hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

# Some constants

SEED = 314159265
VALID_SIZE = 0.2
TARGET = 'outcome'

#-------------------------------------------------#

# Utility functions

def intersect(l_1, l_2):
    return list(set(l_1) & set(l_2))


def get_features(train, test):
    intersecting_features = intersect(train.columns, test.columns)
#     intersecting_features.remove('id')
#     intersecting_features.remove('activity_id')
    return sorted(intersecting_features)


In [4]:

import xgboost as xgb
#-------------------------------------------------#

# Scoring and optimization functions


def score(params):
    print("Training with params: ")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(train_features, label=y_train)
    dvalid = xgb.DMatrix(valid_features, label=y_valid)
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm_model = xgb.train(params, dtrain, num_round,
                          evals=watchlist,
                          feval=gini_xgb,
                          verbose_eval=False)
    predictions = gbm_model.predict(dvalid,
                                    ntree_limit=gbm_model.best_iteration + 1)
    score = eval_gini(y_valid, predictions)
    # TODO: Add the importance for the selected features
    print("\tScore {0}\n\n".format(score))
    # The score function should return the loss (1-score)
    # since the optimize function looks for the minimum
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}


def optimize(
             trials, 
             random_state=SEED):
    """
    This is the optimization function that given a space (space here) of 
    hyperparameters and a scoring function (score here), finds the best hyperparameters.
    """
    # To learn more about XGBoost parameters, head to this page: 
    # https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
    space = {
        'n_estimators': hp.quniform('n_estimators', 100, 1000, 1),
        'eta': hp.quniform('eta', 0.025, 0.1, 0.025),
        # A problem with max_depth casted to float instead of int with
        # the hp.quniform method.
        'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 20, 0.1),
        'reg_alpha':hp.quniform('reg_alpha', 0.5, 15, 1),
        'reg_lambda':hp.quniform('reg_lambda', 0.5, 15, 1),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'objective': 'binary:logistic',
        # Increase this number if you have more cores. Otherwise, remove it and it will default 
        # to the maxium number. 
        'nthread': 4,
        'booster': 'gbtree',
        'tree_method': 'exact',
        'silent': 1,
        'seed': random_state
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest, 
                trials=trials, 
                max_evals=250)
    return best



In [5]:
#-------------------------------------------------#


# Load processed data

# You could use the following script to generate a well-processed train and test data sets:
# https://www.kaggle.com/yassinealouini/predicting-red-hat-business-value/features-processing
# I have only used the .head() of the data sets since the process takes a long time to run.
# I have also put the act_train and act_test data sets since I don't have the processed data sets 
# loaded. 

target = train_label
train_df = pd.DataFrame(X.toarray())
test_df = pd.DataFrame(X_test.toarray())

# train_df = pd.read_csv('../data/act_train.csv').head(100)
# test_df = pd.read_csv('../input/act_test.csv').head(100)

FEATURES = get_features(train_df, test_df)
print(FEATURES)



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [6]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,213,214,215,216,217,218,219,220,221,222
0,2.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,9.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,213,214,215,216,217,218,219,220,221,222
0,0.0,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,7.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [8]:
from sklearn.cross_validation import train_test_split
# Extract the train and valid (used for validation) dataframes from the train_df
train, valid = train_test_split(train_df, test_size=VALID_SIZE,
                                random_state=SEED)

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
trn_idx = train.index
valid_idx = valid.index

In [10]:

#-------------------------------------------------#

train_features = train[FEATURES]
valid_features = valid[FEATURES]
y_train = train_label[trn_idx]
y_valid = train_label[valid_idx]

print('The training set is of length: ', len(train.index))
print('The validation set is of length: ', len(valid.index))

#-------------------------------------------------#

#

The training set is of length:  476169
The validation set is of length:  119043


In [11]:
# Run the optimization

# Trials object where the history of search will be stored
# For the time being, there is a bug with the following version of hyperopt.
# You can read the error messag on the log file.
# For the curious, you can read more about it here: https://github.com/hyperopt/hyperopt/issues/234
# => So I am commenting it.
trials = Trials()

best_hyperparams = optimize(
                            trials
                            )
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)

The best hyperparameters are:  

# {'colsample_bytree': 0.5, 'eta': 0.025, 'gamma': 7.5, 'max_depth': 10, 'min_child_weight': 2.0, 'n_estimators': 819.0, 
#  'reg_alpha': 2.0, 'reg_lambda': 12.0, 'subsample': 0.9500000000000001}

Training with params: 
{'booster': 'gbtree', 'colsample_bytree': 0.8500000000000001, 'eta': 0.05, 'gamma': 1.1, 'max_depth': 3, 'min_child_weight': 5.0, 'n_estimators': 273.0, 'nthread': 4, 'objective': 'binary:logistic', 'reg_alpha': 4.0, 'reg_lambda': 14.0, 'seed': 314159265, 'silent': 1, 'subsample': 0.9, 'tree_method': 'exact'}
	Score 0.28750404800851115


Training with params: 
{'booster': 'gbtree', 'colsample_bytree': 0.9500000000000001, 'eta': 0.025, 'gamma': 8.4, 'max_depth': 4, 'min_child_weight': 2.0, 'n_estimators': 771.0, 'nthread': 4, 'objective': 'binary:logistic', 'reg_alpha': 14.0, 'reg_lambda': 10.0, 'seed': 314159265, 'silent': 1, 'subsample': 0.9500000000000001, 'tree_method': 'exact'}
	Score 0.2879184489956059


Training with params: 
{'booster': 'gbtree', 'colsample_bytree': 0.55, 'eta': 0.1, 'gamma': 7.5, 'max_depth': 6, 'min_child_weight': 2.0, 'n_estimators': 691.0, 'nthread': 4, 'objective': 'binary:logistic', 'reg_alpha': 4.0, 'reg_lambda': 1.0, 'seed': 314159

In [13]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
import gc

import time 
from datetime import datetime

In [ ]:
target = train_label
trn_df = pd.DataFrame(X.toarray())
sub_df = pd.DataFrame(X_test.toarray())

n_splits = 5
n_estimators = 819

final_cv_pred = np.zeros(len(sub_df))
x_score = []

# for sp in range(1,30, 1):
for sp in np.linspace(1., 3., 20):
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=15) 
    imp_df = np.zeros((len(trn_df.columns), n_splits))
    xgb_evals = np.zeros((n_estimators, n_splits))
    oof = np.empty(len(trn_df))
    sub_preds = np.zeros(len(sub_df))
    increase = True

    np.random.seed(0)

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(target, target)):
        trn_dat, trn_tgt = trn_df.iloc[trn_idx], target.iloc[trn_idx]
        val_dat, val_tgt = trn_df.iloc[val_idx], target.iloc[val_idx]


        # Upsample during cross validation to avoid having the same samples
        # in both trn_df and validation sets
        # Validation set is not up-sampled to monitor overfitting
        if increase:
            # Get positive examples
            pos = pd.Series(trn_tgt == 1)
            # Add positive examples
            trn_dat = pd.concat([trn_dat, trn_dat.loc[pos]], axis=0)
            trn_tgt = pd.concat([trn_tgt, trn_tgt.loc[pos]], axis=0)
            # Shuffle data
            idx = np.arange(len(trn_dat))
            np.random.shuffle(idx)
            trn_dat = trn_dat.iloc[idx]
            trn_tgt = trn_tgt.iloc[idx]

    #     print(sum(trn_tgt==0)/sum(trn_tgt==1))

    #     sp = sum(trn_tgt==0)/sum(trn_tgt==1)
    #         sp = 1.55556
#         sp = 1.52631578947
#         clf = XGBClassifier(n_estimators=n_estimators,
#                             max_depth=4,
#                             objective="binary:logistic",
#                             learning_rate=.1, 
#                             subsample=.8, 
#                             colsample_bytree=.8,
#                             scale_pos_weight=sp,
#                             missing=-1,
#                             gamma=1,
#                             reg_alpha=0,
#                             reg_lambda=1,
#                             seed=s,
#                             nthread=5)

        print(sp)
        xgb_param_2 = {
            'learning_rate': 0.07,
            'objective':"binary:logistic",
            'n_estimators':200,
            'max_depth':4,
            'min_child_weight':6,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'scale_pos_weight':sp,
            'gamma':10,
            'reg_alpha':8,
            'reg_lambda':1.3,
            'seed':99
        }
        xgb_param_2 = {'colsample_bytree': 0.5, 'learning_rate': 0.025, 'gamma': 7.5, 
                        'max_depth': 10, 'min_child_weight': 2.0, 'scale_pos_weight':sp,
                        'n_estimators': 819, 'reg_alpha': 2.0, 
                        'reg_lambda': 12.0, 'subsample': 0.95}
        clf = XGBClassifier(** xgb_param_2)
        clf.fit(trn_dat, trn_tgt, 
                eval_set=[(trn_dat, trn_tgt), (val_dat, val_tgt)],
                eval_metric=gini_xgb,
                early_stopping_rounds=None,
                verbose=False)

        # Keep feature importances
        imp_df[:, fold_] = clf.feature_importances_

        # Find best round for validation set
        xgb_evals[:, fold_] = clf.evals_result_["validation_1"]["gini"]
        # Xgboost provides best round starting from 0 so it has to be incremented
        best_round = np.argsort(xgb_evals[:, fold_])[::-1][0]
#         print(best_round)

        # Predict OOF and submission probas with the best round
        oof[val_idx] = clf.predict_proba(val_dat, ntree_limit=int(best_round))[:, 1]
        # Update submission
        sub_preds += clf.predict_proba(sub_df, ntree_limit=int(best_round))[:, 1] / n_splits

        # Display results
        print("Fold %2d : %.6f @%4d / best score is %.6f @%4d"
              % (fold_ + 1,
                 eval_gini(val_tgt, oof[val_idx]),
                 n_estimators,
                 xgb_evals[best_round, fold_],
                 best_round))

    print(sp)
    full_oof_score = eval_gini(target, oof)
    print("Full OOF score : %.6f" % eval_gini(target, oof))
    # Compute mean score and std
    mean_eval = np.mean(xgb_evals, axis=1)
    std_eval = np.std(xgb_evals, axis=1)
    best_round = np.argsort(mean_eval)[::-1][0]

    print("Best mean score : %.6f + %.6f @%4d"
          % (mean_eval[best_round], std_eval[best_round], best_round))
    print('--------------------------------------')
    print('\n')
    m1 = np.min(sub_preds)
    m2 = np.max(sub_preds)
    print(m1, m2)
    assert((m1 > 0) & (m2 <= 1.))
    final_cv_pred += sub_preds
    
    x_score.append(full_oof_score)
print(x_score)
pd.DataFrame({'id': test_id, 'target': final_cv_pred / 16.}).to_csv('../submissions/xgb3_pred_avg.csv', index=False)

1.0
Fold  1 : 0.276190 @ 819 / best score is 0.276197 @ 307
1.0
Fold  2 : 0.280033 @ 819 / best score is 0.280146 @ 437
1.0
Fold  3 : 0.305255 @ 819 / best score is 0.305271 @ 305
1.0
Fold  4 : 0.281575 @ 819 / best score is 0.281577 @ 550
1.0
Fold  5 : 0.280406 @ 819 / best score is 0.280440 @ 281
1.0
Full OOF score : 0.284458
Best mean score : 0.284300 + 0.010566 @ 307
--------------------------------------


0.0159379192628 0.662575341761
1.10526315789
Fold  1 : 0.275597 @ 819 / best score is 0.275606 @ 295
1.10526315789
Fold  2 : 0.278035 @ 819 / best score is 0.278220 @ 437
1.10526315789
Fold  3 : 0.305116 @ 819 / best score is 0.305249 @ 244
1.10526315789
Fold  4 : 0.280915 @ 819 / best score is 0.280995 @ 346
1.10526315789
Fold  5 : 0.277959 @ 819 / best score is 0.277999 @ 352
1.10526315789
Full OOF score : 0.283090
Best mean score : 0.283126 + 0.010766 @ 303
--------------------------------------


0.0177096952684 0.690811738372
1.21052631579
Fold  1 : 0.275919 @ 819 / best sc

In [ ]:
print(sum(x_score)/16)

In [ ]:
np.max(final_cv_pred
      )